# Export shot transitions
Run this notebook from an esper movies environment.

In [1]:
import pickle
import numpy as np
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.interval_list import IntervalList
from rekall.temporal_predicates import equal, overlaps
import matplotlib.pyplot as plt
from query.models import LabeledInterval, Labeler, Shot
import os
from tqdm import tqdm
from esper.prelude import *
from PIL import Image

# Load Shot Transitions

In [2]:
shots_qs = Shot.objects.filter(labeler__name__contains="manual")

In [3]:
shots = VideoIntervalCollection.from_django_qs(shots_qs)

In [4]:
window_size = 16
stride = 16

shot_boundaries = shots.map(
    lambda intrvl: (intrvl.start, intrvl.start, intrvl.payload)
).set_union(
    shots.map(lambda intrvl: (intrvl.end + 1, intrvl.end + 1, intrvl.payload))
).coalesce().filter(lambda intrvl: intrvl.payload != -1)

clips = shots.dilate(1).coalesce().dilate(-1).map(
    lambda intrvl: (
        intrvl.start - stride - ((intrvl.start - stride) % stride),
        intrvl.end + stride - ((intrvl.end + stride) % stride),
        intrvl.payload
    )
).dilate(1).coalesce().dilate(-1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 51599.52it/s]


In [5]:
VAL_WINDOWS = '/app/data/shot_detection_weak_labels/validation_windows_same_val_test.pkl'
TEST_WINDOWS = '/app/data/shot_detection_weak_labels/test_windows_same_val_test.pkl'

In [6]:
with open(VAL_WINDOWS, 'rb') as f:
    val_windows_by_video_id = pickle.load(f)
with open(TEST_WINDOWS, 'rb') as f:
    test_windows_by_video_id = pickle.load(f)

In [7]:
shot_boundary_tuples = [
    (video_id, intrvl.start)
    for video_id in shot_boundaries.get_allintervals()
    for intrvl in shot_boundaries.get_intervallist(video_id).get_intervals()
]

In [8]:
val_frames = [
    (video_id, frame_number, 1 if (video_id, frame_number) in shot_boundary_tuples else 0)
    for video_id, window_start, window_end in val_windows_by_video_id
    for frame_number in range(window_start, window_end)
]

In [9]:
test_frames = [
    (video_id, frame_number, 1 if (video_id, frame_number) in shot_boundary_tuples else 0)
    for video_id, window_start, window_end in test_windows_by_video_id
    for frame_number in range(window_start, window_end)
]

# Export frames

In [10]:
all_frame_numbers = val_frames + test_frames

In [11]:
frame_numbers_by_video = {}
for video_id, frame_number, label in all_frame_numbers:
    if video_id not in frame_numbers_by_video:
        frame_numbers_by_video[video_id] = []
    frame_numbers_by_video[video_id].append(frame_number)
for video_id in frame_numbers_by_video:
    frame_numbers_by_video[video_id] = sorted(frame_numbers_by_video[video_id])

In [12]:
import hwang, storehouse

In [13]:
root_path = '/app/data'

In [14]:
Video.objects.get(id=1).path

'movies/12_years_a_slave_2013.mp4'

In [16]:
for video_id in frame_numbers_by_video:
    print(video_id, len(frame_numbers_by_video[video_id]))

65 1344
515 608
577 1360
201 1344
226 1136
557 720
144 640
504 1424
339 1088
148 1184
23 1344
411 608
34 1216
123 21328
359 1328
104 1376
172 1392
370 7504
178 1216
179 1344
308 736
181 1104
54 1376
585 1360
248 1008
116 1376
315 976
574 1168


In [19]:
for video_id in frame_numbers_by_video:
    if video_id == 123:
        continue
    print(video_id)
    video = Video.objects.get(id=video_id)
    backend = storehouse.StorageBackend.make_from_config(
        storehouse.StorageConfig.make_posix_config()
    )
    video_path = os.path.join(root_path, video.path)
    dec = hwang.Decoder(storehouse.RandomReadFile(backend, video_path))
    
    frame_nums = frame_numbers_by_video[video_id]
    os.makedirs('/app/data/shot_transitions/images/{}'.format(video_id), exist_ok=True)
    
    for i in tqdm(list(range(0, len(frame_nums), 500))):
        cur_frame_nums = frame_nums[i:i+500]
    
        frames = dec.retrieve(cur_frame_nums)

        for frame_num, frame in zip(cur_frame_nums, frames):
            im = Image.fromarray(frame)
            im.save('/app/data/shot_transitions/images/{}/{:06d}.jpg'.format(
                video_id, frame_num))
    
        del frames

65


515


577


201


226


...

585


248


116


315


574


# Export train/val/test splits

In [22]:
val_frames

[(23, 79920, 0),
 (23, 79921, 0),
 (23, 79922, 0),
 (23, 79923, 0),
 (23, 79924, 0),
 (23, 79925, 0),
 (23, 79926, 0),
 (23, 79927, 0),
 (23, 79928, 0),
 (23, 79929, 0),
 (23, 79930, 0),
 (23, 79931, 0),
 (23, 79932, 0),
 (23, 79933, 0),
 (23, 79934, 1),
 (23, 79935, 0),
 (23, 79936, 0),
 (23, 79937, 0),
 (23, 79938, 0),
 (23, 79939, 0),
 (23, 79940, 0),
 (23, 79941, 0),
 (23, 79942, 0),
 (23, 79943, 0),
 (23, 79944, 0),
 (23, 79945, 0),
 (23, 79946, 0),
 (23, 79947, 0),
 (23, 79948, 0),
 (23, 79949, 0),
 (23, 79950, 0),
 (23, 79951, 0),
 (23, 79952, 0),
 (23, 79953, 0),
 (23, 79954, 0),
 (23, 79955, 0),
 (23, 79956, 0),
 (23, 79957, 0),
 (23, 79958, 0),
 (23, 79959, 0),
 (23, 79960, 0),
 (23, 79961, 0),
 (23, 79962, 0),
 (23, 79963, 0),
 (23, 79964, 0),
 (23, 79965, 0),
 (23, 79966, 0),
 (23, 79967, 0),
 (23, 79968, 0),
 (23, 79969, 0),
 (23, 79970, 0),
 (23, 79971, 0),
 (23, 79972, 0),
 (23, 79973, 0),
 (23, 79974, 0),
 (23, 79975, 0),
 (23, 79976, 0),
 (23, 79977, 0),
 (23, 79978, 0

In [24]:
os.makedirs('/app/data/shot_transitions/data', exist_ok=True)
with open('/app/data/shot_transitions/data/val.txt', 'w') as f:
    for video_id, frame, label in val_frames:
        f.write('{} {} {}\n'.format(video_id, frame, label))
with open('/app/data/shot_transitions/data/test.txt', 'w') as f:
    for video_id, frame, label in test_frames:
        f.write('{} {} {}\n'.format(video_id, frame, label))